In [33]:
import cv2
import time
import datetime
import imutils
import gym
import random
import matplotlib.pyplot as plt
from tqdm import tqdm
from PIL import Image


In [34]:
env = gym.make('VideoPinball-v0', difficulty=1)
height, width, channels = env.observation_space.shape
actions = env.action_space.n
print(actions)

9


In [35]:
env.unwrapped.get_action_meanings()

env.reset()

img = env.render(mode='rgb_array')
# rgb = Image.fromarray(img)
cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [59]:
first_frame = None
env.reset()
score = 0
rendered = None
xs = []
ys = []

for i in tqdm(range(1000)):
    frame = env.render(mode='rgb_array')
    # plt.imshow(frame)
    # plt.show()
    action = random.choice([3,4])
    n_state, reward, done, info = env.step(action)
    greyscale_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gaussian_frame = cv2.GaussianBlur(greyscale_frame, (21,21),0)
    blur_frame = cv2.blur(gaussian_frame, (5,5))
    greyscale_image = blur_frame
    # plt.imshow(frame)
    # plt.show()
    # plt.imshow(greyscale_frame)
    # plt.show()
    # plt.imshow(gaussian_frame)
    # plt.show()
    # plt.imshow(blur_frame)
    # plt.show()
    if first_frame is None:
        first_frame = greyscale_image
    else:
        pass
    frame_delta = cv2.absdiff(first_frame, greyscale_image)
    # plt.imshow(frame_delta)
    # plt.show()

    thresh = cv2.threshold(frame_delta, 30, 255, cv2.THRESH_BINARY)[1]
    # plt.imshow(thresh)
    # plt.show()
    dilate_image = cv2.dilate(thresh, None, iterations=2)
    # plt.imshow(dilate_image)
    # plt.show()
    cnts = cv2.findContours(dilate_image.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]
    if cnts is not None:
        # print(cnts)
        for c in cnts:
            # print(c)
            if cv2.contourArea(c) > 10:
                (x, y, w, h) = cv2.boundingRect(c)
                xs.append(x)
                xs.append(x+w)
                ys.append(y)
                ys.append(y+h)
            else:
                pass

env.close()
print(min(xs))
print(max(xs))
print(min(ys))
print(max(ys))

100%|██████████| 1000/1000 [00:03<00:00, 266.53it/s]

64
96
174
196
